In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'/home/ajith/projects/Multivideo_Objectdetection_MLOPS_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from objectDetection.constants import *
from objectDetection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config

In [5]:
import os 
import urllib.request as request
import zipfile
from objectDetection import logger
from objectDetection.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-04 10:33:57,642: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-04 10:33:57,645: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-04 10:33:57,647: INFO: common: created directory at: artifacts]
[2024-02-04 10:33:57,650: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-04 10:34:02,851: INFO: 2306223797: artifacts/data_ingestion/dataset.zip download! with following info: 
 Connection: close
Content-Length: 54891690
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "36f98115295b0ba94d6e877234607de1de47bddd2c8ab0885136eea9c8a4f9a1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 35D8:24B3E5:15E194:220F49:65BF1ABC
Accept-Ranges: bytes
Date: Sun, 04 Feb 2024 05:03:59 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100033-HYD
X-C